In [ ]:
#  딕셔너리 키로 선언 : 경력 0년=신입 
# 문제 발생, 해결책 : 사전 크기가 변경 / 사전에서 직접 수정하는 방법 대신, 모든 키를 다른 리스트에 저장한 후 별도로 사전에서 수정하도록 변경하고 누락된 직무 확인하는 코드 추가 

경력 0년차를 "신입"으로 변환하는 코드가 문제를 일으킬 가능성이 있습니다. 특히 for duty in job_career_dict.keys(): 같은 루프에서 keys()를 직접 순회할 경우, 사전 크기가 동적으로 변경되면 예상치 못한 동작이 발생할 수 있습니다.
즉, 0을 신입으로 바꾸면서 직무가 누락될 수 있습니다.

In [ ]:
✅ 해결 방법
키 변경 작업을 별도 리스트에 저장 후, 원래 사전에서 수정
→ dict.keys()를 직접 변경하면 순회 중 오류가 발생할 수 있으므로, 키를 미리 저장해 두고 업데이트해야 합니다.

직무가 누락되는 원인을 파악하기 위해 디버깅 코드 추가
→ job_career_dict의 크기를 비교하고, duty_analysis 테이블과 비교해 어떤 직무가 빠졌는지 확인.

In [ ]:
✅ 이 코드의 주요 수정 사항
"경력 0년차"를 "신입"으로 변환하는 과정에서 사전 크기 변경 문제 해결

job_career_dict.keys()를 순회하면서 직접 수정하는 방식 대신,
모든 키를 duties_to_update 리스트에 저장한 후, 별도로 job_career_dict를 수정하도록 변경.
누락된 직무를 확인하는 코드 추가

duty_analysis에 있는 직무와 job_career_dict에 있는 직무를 비교해서 어떤 직무가 누락되었는지 출력.
연차별 데이터가 없는 직무는 건너뛰고 경고 메시지 출력

if duty not in job_career_dict: 조건을 체크하여 해당 직무에 데이터가 없으면 continue.


In [ ]:
import openai
import sqlite3


In [2]:
# ✅ SQLite 데이터베이스 연결
db_path = "asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ `duty_analysis` 테이블에서 career 데이터 조회
cursor.execute("SELECT duty, career FROM duty_analysis;")
rows = cursor.fetchall()

# ✅ 직무별 연차별 채용 데이터 조회
cursor.execute("""
    SELECT duty, career, COUNT(*) as job_count
    FROM processing
    GROUP BY duty, career
    ORDER BY duty, career;
""")
career_data = cursor.fetchall()

# ✅ 데이터 변환 (직무별 연차별 데이터 정리)
job_career_dict = {}
for duty, career, job_count in career_data:
    if duty not in job_career_dict:
        job_career_dict[duty] = {}
    job_career_dict[duty][career] = job_count

# ✅ "경력 0년차"를 "신입"으로 변경 (안전하게 처리)
duties_to_update = list(job_career_dict.keys())  # 키 리스트를 별도 저장

for duty in duties_to_update:
    if 0 in job_career_dict[duty]:  # 0년차 데이터가 존재하면
        job_career_dict[duty]["신입"] = job_career_dict[duty].pop(0)  # "신입" 키로 이동 후 기존 0 삭제

# ✅ 모든 직무가 정상적으로 포함되었는지 확인
cursor.execute("SELECT DISTINCT duty FROM duty_analysis;")
all_duties = {row[0] for row in cursor.fetchall()}  # 데이터베이스의 모든 직무
processed_duties = set(job_career_dict.keys())  # 처리된 직무

missing_duties = all_duties - processed_duties
if missing_duties:
    print(f"❌ 누락된 직무: {missing_duties}")
else:
    print("🎯 모든 직무가 정상적으로 처리되었습니다!")

# ✅ 직무별 GPT-3.5 요약 생성 및 업데이트
for duty, career in rows:
    if duty not in job_career_dict:
        print(f"⚠️ {duty} 직무의 연차별 데이터가 없음 (건너뜀)")
        continue  # 해당 직무에 대한 데이터가 없으면 건너뜀

    # ✅ 프롬프트 생성
    job_details = "\n".join([f"- {key}년차: {value}건" if key != "신입" else f"- 신입: {value}건"
                              for key, value in job_career_dict[duty].items()])
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 개수]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 많이 요청된 경력 연차와 해당 채용 공고 수
    2. 두 번째로 많이 요청된 경력 연차와 해당 채용 공고 수
    3. 최소 요구되는 연차
    4. {duty} 직무에서의 경력별 채용 특징 및 트렌드 분석

    ❗ 단, "신입"이라는 표현을 사용하며, "경력 0년차"라는 표현은 하지 마세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요. 
    """

    try:
        # ✅ GPT-3.5 API 호출
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt3_summary = response.choices[0].message.content.strip()

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
        existing_career_data = cursor.fetchone()

        if existing_career_data and existing_career_data[0].endswith(".png"):
            image_path = existing_career_data[0]  # 이미지 경로 유지
        else:
            image_path = ""

        # ✅ `career` 값 업데이트 (['이미지 경로', 'GPT-3.5 설명'])
        career_list = [image_path, gpt3_summary] if image_path else [gpt3_summary]
        updated_text = str(career_list)  # 리스트를 문자열로 변환

        cursor.execute(
            "UPDATE duty_analysis SET career = ? WHERE duty = ?;",
            (updated_text, duty)
        )
        conn.commit()

        print(f"✅ {duty} 요약 완료 (GPT-3.5 적용): {gpt3_summary}")

    except Exception as e:
        print(f"❌ {duty} 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()
print("🎯 모든 직무의 경력 요구 사항 요약이 GPT-3.5로 업데이트되었습니다.")


❌ 누락된 직무: {'VR'}
✅ AI 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 38건입니다.

2. 두 번째로 많이 요청된 경력 연차는 2년차이며, 해당 채용 공고 수는 25건입니다.

3. AI 직무에서의 최소 요구되는 연차는 신입이며, 해당 채용 공고 수는 23건입니다.

4. AI 직무에서는 3년차와 2년차가 가장 많이 요구되는 경력 연차로, 경력이 쌓이면서 채용 공고 수가 감소하는 경향을 보입니다. 신입의 경우에도 많은 기회가 제공되고 있으며, 경력이 있는 지원자뿐만 아니라 신입 지원자들에게도 기회가 많이 주어지고 있다는 흐름이 나타나고 있습니다.
✅ ERP 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 7년차이며, 해당 채용 공고 수는 3건입니다.

2. 두 번째로 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 3건입니다.

3. 최소 요구되는 연차는 1년차이며, 해당 채용 공고 수는 1건입니다.

4. ERP 직무에서는 경력 연차가 높을수록 요구되는 책임과 업무 범위가 넓어지는 경향이 있습니다. 또한, 신입으로의 채용은 적은 편이지만, 중간 경력층에 대한 수요도 존재하는 것으로 보입니다.
✅ PM 요약 완료 (GPT-3.5 적용): 1. 가장 많이 요청된 경력 연차는 5년차이며, 해당 채용 공고 수는 22건입니다.

2. 두 번째로 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 14건입니다.

3. PM 직무에서 최소 요구되는 연차는 신입이며, 해당 채용 공고 수는 8건입니다.

4. PM 직무에서는 주로 5년차 경력이 요구되며, 3년차 및 신입 역시 많은 수요가 있습니다. 또한, 1년차, 2년차 등으로 진입하기 어려운 트렌드를 보이고 있습니다.
✅ QA 요약 완료 (GPT-3.5 적용): 1. QA 직무에서 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 39건입니다.

2. QA 직무에서 두 번째로 많이 요청된 경력

In [5]:
import sqlite3
import ast  # 문자열 리스트 변환용

# ✅ SQLite 데이터베이스 연결
db_path = "asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ VR 직무의 career 컬럼 데이터 조회
cursor.execute("SELECT career FROM duty_analysis WHERE duty = 'VR';")
row = cursor.fetchone()

# ✅ 2번째 원소 값 출력
if row and row[0]:  # 데이터가 존재하는 경우
    try:
        # 문자열로 저장된 리스트를 실제 리스트로 변환
        career_list = ast.literal_eval(row[0])

        # 리스트가 있고, 최소 2개의 원소가 있을 경우 출력
        if isinstance(career_list, list) and len(career_list) > 1:
            print(f"📌 VR 직무의 2번째 값: {career_list[1]}")
        else:
            print("⚠️ VR 직무의 career 데이터가 부족합니다.")
    except Exception as e:
        print(f"❌ VR 직무 데이터 변환 오류: {e}")
else:
    print("❌ VR 직무 데이터가 없습니다.")

# ✅ DB 연결 종료
conn.close()


📌 VR 직무의 2번째 값: 1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 5건입니다. 2. 두 번째로 많이 요청된 경력 연차는 0년차, 2년차, 5년차로 같은 수치인 3건입니다. 3. 최소 요구되는 연차는 0년차입니다. 4. VR 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 경력이 높아질수록 채용 공고 수가 감소하는 경향이 있습니다.    - 가장 많이 요구된 경력 연차는 3년차로, 공고 수도 가장 많았습니다. 이는 VR 분야에서 중간 정도의 경력을 가진 인력을 선호한다는 특징을 보여줍니다.    - 또한, 최소 요구되는 연차가 0년차인 것을 보아 VR 분야는 신입 인력을 공급받는 비율도 높습니다.    - VR 직무는 신입부터 중간 경력자까지 다양한 경력 레벨을 뽑는 경향이 있어 경력이 다양해도 기회가 많아 보입니다.


In [6]:
import sqlite3
import ast  # 문자열을 리스트로 변환하는 용도

# ✅ SQLite 데이터베이스 연결
db_path = "asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ VR 직무의 career 데이터 가져오기
cursor.execute("SELECT career FROM duty_analysis WHERE duty = 'VR';")
vr_career_data = cursor.fetchone()

if vr_career_data:
    try:
        # ✅ 문자열로 저장된 career 데이터를 리스트로 변환
        career_list = ast.literal_eval(vr_career_data[0])

        # ✅ 리스트에 최소 두 개의 원소가 존재하는지 확인
        if isinstance(career_list, list) and len(career_list) > 1:
            # ✅ 2번째 원소(설명)에서 "0년차"를 "신입"으로 강제 변경
            modified_text = career_list[1].replace("0년차", "신입")

            # ✅ 변경된 내용을 리스트에 반영
            career_list[1] = modified_text

            # ✅ `duty_analysis` 테이블의 `career` 컬럼 업데이트
            cursor.execute(
                "UPDATE duty_analysis SET career = ? WHERE duty = 'VR';",
                (str(career_list),)
            )
            conn.commit()

            print("✅ VR 직무의 '0년차'가 '신입'으로 변환되어 DB에 반영되었습니다.")
        else:
            print("⚠️ VR 직무의 career 데이터가 부족합니다.")
    except Exception as e:
        print(f"❌ 데이터 변환 오류: {e}")
else:
    print("⚠️ VR 직무의 데이터가 존재하지 않습니다.")

# ✅ DB 연결 종료
conn.close()


✅ VR 직무의 '0년차'가 '신입'으로 변환되어 DB에 반영되었습니다.


In [7]:
import sqlite3
import ast  # 문자열 리스트 변환용

# ✅ SQLite 데이터베이스 연결
db_path = "asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ VR 직무의 career 컬럼 데이터 조회
cursor.execute("SELECT career FROM duty_analysis WHERE duty = 'VR';")
row = cursor.fetchone()

# ✅ 2번째 원소 값 출력
if row and row[0]:  # 데이터가 존재하는 경우
    try:
        # 문자열로 저장된 리스트를 실제 리스트로 변환
        career_list = ast.literal_eval(row[0])

        # 리스트가 있고, 최소 2개의 원소가 있을 경우 출력
        if isinstance(career_list, list) and len(career_list) > 1:
            print(f"📌 VR 직무의 2번째 값: {career_list[1]}")
        else:
            print("⚠️ VR 직무의 career 데이터가 부족합니다.")
    except Exception as e:
        print(f"❌ VR 직무 데이터 변환 오류: {e}")
else:
    print("❌ VR 직무 데이터가 없습니다.")

# ✅ DB 연결 종료
conn.close()


📌 VR 직무의 2번째 값: 1. 가장 많이 요청된 경력 연차는 3년차이며, 해당 채용 공고 수는 5건입니다. 2. 두 번째로 많이 요청된 경력 연차는 신입, 2년차, 5년차로 같은 수치인 3건입니다. 3. 최소 요구되는 연차는 신입입니다. 4. VR 직무에서의 경력별 채용 특징 및 트렌드 분석:    - 경력이 높아질수록 채용 공고 수가 감소하는 경향이 있습니다.    - 가장 많이 요구된 경력 연차는 3년차로, 공고 수도 가장 많았습니다. 이는 VR 분야에서 중간 정도의 경력을 가진 인력을 선호한다는 특징을 보여줍니다.    - 또한, 최소 요구되는 연차가 신입인 것을 보아 VR 분야는 신입 인력을 공급받는 비율도 높습니다.    - VR 직무는 신입부터 중간 경력자까지 다양한 경력 레벨을 뽑는 경향이 있어 경력이 다양해도 기회가 많아 보입니다.
